In [4]:
# converting torch model to onnx
import os
import torch
import torch.nn as nn
from models import *
from IPython.display import Markdown, display

device = "cuda" if torch.cuda.is_available() else "cpu"


In [5]:
# Parameters
ONNX_SAVE_DIR_PATH = "trained_models/onnx/"
MODEL_PATH = "trained_models/its_train_ffa_3_9.pk"
IMAGE_HEIGHT = 240 #crop size
IMAGE_WIDTH = 240 #crop size
IR_MODEL_DIR = "trained_models/IR/"

In [6]:
def make_model(MODEL_PATH):
    _MODEL_NAME = MODEL_PATH.split('/')[-1]

    _MODEL_DATASET = _MODEL_NAME.split('_')[0]
    _MODEL_MODE = _MODEL_NAME.split('_')[1]
    _MODEL_ARCH = _MODEL_NAME.split('_')[2]
    _MODEL_NO_GROUPS = _MODEL_NAME.split('_')[3]
    _MODEL_NO_BLOCKS = _MODEL_NAME.split('_')[4]

    if _MODEL_ARCH == 'ffa':
        return FFA(gps = _MODEL_NO_GROUPS,blocks = _MODEL_NO_BLOCKS)
    print('models other than FFA not yet implemented')
    return None

In [7]:
MODEL_PATH = "trained_models/its_train_ffa_3_9.pk"
# dehaze_net = make_model(MODEL_PATH) # dynamic initialization #!TODO
dehaze_net = FFA(gps =3,blocks=9) # Manually initialize FFA net
dehaze_net = dehaze_net.to(device)
checkpoint = torch.load(MODEL_PATH)
dehaze_net.load_state_dict(checkpoint["model"])

<All keys matched successfully>

In [8]:
dummy_input = torch.randn(1, 3, 240, 240).to(device)
model_name = MODEL_PATH.split('/')[-1].split('.')[0]
onnx_model_save_path = ONNX_SAVE_DIR_PATH+ '/' + model_name + '.pth'

# torch to onnx
torch.onnx.export(
        dehaze_net,
        dummy_input,
        onnx_model_save_path,
        opset_version=11,
        do_constant_folding=False,
)

## Convert ONNX model to OpenVINO IR Format

In [9]:
# Construct the command for Model Optimizer
mo_command = f"""mo
                 --input_model "{onnx_model_save_path}"
                 --input_shape "[1,3, {IMAGE_HEIGHT}, {IMAGE_WIDTH}]"
                 --data_type FP16
                 --output_dir "{IR_MODEL_DIR}"
                 """
mo_command = " ".join(mo_command.split())
print("Model Optimizer command to convert the ONNX model to OpenVINO:")
display(Markdown(f"`{mo_command}`"))

Model Optimizer command to convert the ONNX model to OpenVINO:


`mo --input_model "trained_models/onnx//its_train_ffa_3_9.pth" --input_shape "[1,3, 240, 240]" --data_type FP16 --output_dir "trained_models/IR/"`

In [10]:
print("Exporting ONNX model to IR... This may take a few minutes.")
mo_result = %sx $mo_command
print("\n".join(mo_result))

Exporting ONNX model to IR... This may take a few minutes.
'mo' is not recognized as an internal or external command,
operable program or batch file.


In [3]:

from openvino.inference_engine import IECore

ImportError: DLL load failed while importing ie_api: The specified module could not be found.